<a href="https://colab.research.google.com/github/bkristensen/Colabs/blob/main/Billed%20analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔧 INSTALLATION (kun første gang)
!pip install torch torchvision pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

📂 Sådan bruger du det:

	1.	Gå til Google Colab
	2.	Opret en ny Python 3-notesbog
	3.	Indsæt hele scriptet i en celle
	4.	Upload dine billeder som en .zip-fil når scriptet spørger
	5.	Resultaterne vises som: filnavn | beskrivelse | farve


In [ ]:
from google.colab import files
import zipfile
import os

In [ ]:
# 🧠 IMPORTÉR MODELLER OG KLASSER
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
from pathlib import Path
import urllib.request

In [ ]:
# 🎨 Funktion til at finde temafarve
def get_main_color(image):
    image = image.resize((50, 50))
    arr = np.array(image).reshape(-1, 3)
    mean = np.mean(arr, axis=0)
    return tuple(int(x) for x in mean)

In [ ]:
# 📁 UPLOAD DINE BILLEDER
uploaded = files.upload()
for name in uploaded:
    if name.endswith(".zip"):
        with zipfile.ZipFile(name, 'r') as zip_ref:
            zip_ref.extractall("billeder")
        print(f"Udpakket: {name}")

Saving Komprimeret arkiv.zip to Komprimeret arkiv.zip
Udpakket: Komprimeret arkiv.zip


In [ ]:
# 🔤 Hent ImageNet klassenavne
if not os.path.exists("imagenet_classes.txt"):
    urllib.request.urlretrieve(
        "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt",
        "imagenet_classes.txt"
    )
with open("imagenet_classes.txt") as f:
    labels = [line.strip() for line in f.readlines()]

In [ ]:
# 🔄 Forbered model og transform
model = models.resnet50(pretrained=True)
model.eval()
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 132MB/s]


In [ ]:
# 🔍 ANALYSE AF BILLEDER
input_folder = Path("billeder")
results = []

for file in input_folder.glob("**/*"):
    if not file.is_file():
        continue
    if file.stem.startswith("."):
        continue
    if file.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
        continue
    try:
        img = Image.open(file).convert("RGB")
        tensor = transform(img).unsqueeze(0)
        with torch.no_grad():
            output = model(tensor)
            _, pred = output.max(1)
            label = labels[pred.item()]
        try:
           color = get_main_color(img)
        except Exception as e:
           color = str(e)
        results.append((file.name, label, color))
    except Exception as e:
        results.append((file.name, "FEJL", str(e)))


In [19]:
# 📄 VIS RESULTAT
cvs_lines = [("Filnavn", "Label", "Farve")]
for filnavn, label, farve in results:
    cvs_lines.append((filnavn, label, farve))
    # print(f"{filnavn:<30} | {label:<30} | Farve: {farve}")

cvs_file = Path("resultater.csv")
with cvs_file.open("w") as fm:
  fm.writelines([(",").join(str(cell) for cell in line) for line in cvs_lines])
  #for line in cvs_lines:
    # print(line)
  #  fm.write(",".join(str(cell)) for cell in line)
#cvs_file.write_text("/n".join(",".join(str(cell) for cell in line) for line in cvs_lines))
print(cvs_lines)

[('Filnavn', 'Label', 'Farve'), ('IMG_0445.jpg', 'brassiere', (159, 127, 111)), ('IMG_6335.jpg', 'nipple', (138, 116, 99)), ('d63550c9-0b0b-4163-b28d-091f8442e17e.jpg', 'toilet tissue', (141, 124, 113)), ('IMG_2930.jpg', 'shower cap', (112, 116, 106)), ('IMG_3105.jpg', 'nipple', (144, 115, 91)), ('IMG_4371.jpg', 'cradle', (162, 109, 78)), ('IMG_4401.jpg', 'cradle', (134, 104, 78)), ('IMG_0983.jpg', 'bib', (137, 105, 94)), ('IMG_2223.jpg', 'mosquito net', (130, 97, 98)), ('IMG_2721.jpg', 'cradle', (146, 144, 145)), ('IMG_0197.jpg', 'mosquito net', (157, 139, 122)), ('IMG_7699.jpg', 'bib', (149, 116, 136)), ('IMG_6253.jpg', 'cradle', (122, 100, 97)), ('IMG_4426.jpg', 'mosquito net', (156, 112, 76)), ('IMG_2719.jpg', 'cradle', (114, 112, 114)), ('IMG_4963.jpg', 'bassinet', (115, 103, 94)), ('IMG_0299.jpg', 'bassinet', (148, 107, 87)), ('IMG_2004.jpg', 'oxygen mask', (129, 94, 74)), ('IMG_9199.jpg', 'balance beam', (153, 148, 136)), ('2c58a4d3-14ca-4dcc-ae6f-52d05ca1d0b5.jpg', 'cradle', (1